In [0]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier

train = "https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/pump_train_for_models%20(1).csv" 
test =  "https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/water_table_test.csv"
train = pd.read_csv(train)
test = pd.read_csv(test)

In [32]:
test

,id,amount_tsh,days_since_recorded,funder,installer,basin,population,public_meeting,scheme_management,permit,construction_year,management_group,payment,water_quality,quantity,source_class,waterpoint_type
0,50785,0.0,302,other,other,Internal,321,True,Parastatal,True,10s,parastatal,never pay,soft,seasonal,surface,other
1,51630,0.0,302,gov,dwe,Pangani,300,True,vwc,True,00s,user-group,never pay,soft,insufficient,groundwater,communal standpipe
2,17168,0.0,305,other,other,Internal,500,True,vwc,Unknown,10s,user-group,never pay,soft,insufficient,surface,other
3,45559,0.0,315,other,other,Ruvuma / Southern Coast,250,Unknown,vwc,True,80s,user-group,unknown,soft,dry,groundwater,other
4,49871,500.0,251,other,other,Ruvuma / Southern Coast,60,Unknown,wtr_brd,True,00s,user-group,pay monthly,soft,enough,groundwater,communal standpipe
5,52449,0.0,274,gov,dwe,Pangani,200,True,vwc,True,90s,user-group,never pay,soft,enough,groundwater,communal standpipe
6,24806,0.0,1007,gov,other,Rufiji,600,True,vwc,True,00s,user-group,never pay,salty,enough,groundwater,hand pump
7,28965,0.0,312,other,other,Ruvuma / Southern Coast,1,True,wtr_brd,True,80s,user-group,never pay,soft,dry,groundwater,communal standpipe multiple
8,36301,30.0,314,Unicef,other,Ruvuma / Southern Coast,40,True,vwc,False,90s,user-group,pay per bucket,soft,insufficient,groundwater,communal standpipe
9,54122,0.0,260,other,other,Pangani,1,True,wtr_brd,True,00s,user-group,pay monthly,soft,enough,groundwater,communal standpipe


In [0]:
dummy_cols = ['funder', 'installer', 'basin', 'public_meeting', 'scheme_management', 'permit',
              'construction_year', 'management_group', 'payment', 'water_quality',
              'quantity', 'source_class', 'waterpoint_type']

train = pd.get_dummies(train, columns = dummy_cols)

train = train.sample(frac=1).reset_index(drop=True)

In [0]:
test = pd.get_dummies(test, columns = dummy_cols)
#test

In [0]:
target = train.status_group
features = train.drop('status_group', axis=1)

X_train, X_val, y_train, y_val = train_test_split(features, target, train_size=0.8)

In [7]:
target

0                     functional
1                     functional
2                 non functional
3                     functional
4                     functional
5                 non functional
6                     functional
7                     functional
8                     functional
9                     functional
10                non functional
11                    functional
12                    functional
13                    functional
14                    functional
15                    functional
16                non functional
17                non functional
18                    functional
19                non functional
20       functional needs repair
21                non functional
22                    functional
23                    functional
24                    functional
25                non functional
26                non functional
27                    functional
28                non functional
29       functional needs repair
          

In [46]:
xgb = XGBClassifier(n_estimators=250,max_depth=10)
xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
xgb_model = xgb.fit(X_train,y_train)

In [48]:
val_acc = xgb_model.score(X_val,y_val)
val_acc

0.7896464646464646

In [49]:
scores = xgb_model.score(X_train,y_train)
scores

0.8956228956228957

In [50]:
pred = xgb_model.predict(X_train)
pred

array(['functional', 'non functional', 'functional', ..., 'functional',
       'functional', 'functional'], dtype=object)

In [51]:
ytrain=y_train.values
ytrain

array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'functional'], dtype=object)

In [0]:
from sklearn import metrics

In [53]:
metrics.confusion_matrix(ytrain,pred)

array([[25128,    96,   638],
       [ 1190,  2003,   281],
       [ 2686,    69, 15429]])

In [0]:
from sklearn.metrics import classification_report

In [55]:
from sklearn.metrics import classification_report
#target_names = ['functional', 'functional needs repair', 'non functional']
print(classification_report(ytrain, pred))

                         precision    recall  f1-score   support

             functional       0.87      0.97      0.92     25862
functional needs repair       0.92      0.58      0.71      3474
         non functional       0.94      0.85      0.89     18184

            avg / total       0.90      0.90      0.89     47520



In [59]:
from sklearn.externals import joblib
filename = 'xgb_modelmodel.sav'
joblib.dump(xgb_model,filename)

['xgb_modelmodel.sav']

In [0]:
loaded_model_xgb = joblib.load(filename)

In [19]:
loaded_model_xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=nan, n_estimators=250,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
final_xgb=loaded_model_xgb.fit(features, target)     
predictions = loaded_model_xgb.predict(test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
final_filename = 'final_xgb1.sav'
joblib.dump(final_xgb,final_filename)

['final_xgb1.sav']

In [22]:
final_loaded = joblib.load(final_filename)
final_loaded

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=nan, n_estimators=250,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [23]:
predictions

array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [56]:
importances = final_loaded.feature_importances_
importances

array([0.02287708, 0.03063203, 0.1541295 , 0.04866227, 0.00252036,
       0.00756107, 0.00562233, 0.00096937, 0.0073672 , 0.00096937,
       0.0029081 , 0.00077549, 0.00116324, 0.00193874, 0.00445909,
       0.00620396, 0.00426522, 0.00794882, 0.00814269, 0.0073672 ,
       0.00116324, 0.00988755, 0.        , 0.0029081 , 0.00310198,
       0.0065917 , 0.00019387, 0.00756107, 0.00348972, 0.00542846,
       0.00717332, 0.00116324, 0.01085692, 0.00775494, 0.01085692,
       0.00542846, 0.01783637, 0.01143854, 0.00484684, 0.00872431,
       0.02210159, 0.01124467, 0.00329585, 0.00756107, 0.01163242,
       0.00911206, 0.00988755, 0.0036836 , 0.00678558, 0.0036836 ,
       0.00271423, 0.00504071, 0.00445909, 0.01202016, 0.0065917 ,
       0.00678558, 0.01783637, 0.00833656, 0.0110508 , 0.01531601,
       0.00756107, 0.02132609, 0.00484684, 0.00794882, 0.00891819,
       0.00872431, 0.0102753 , 0.00949981, 0.00310198, 0.        ,
       0.        , 0.01415277, 0.01202016, 0.00116324, 0.00484

In [57]:
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_val.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    print(X_val.columns[indices[f]],end=',')
    print()

Feature ranking:
1. feature 2 (0.154130)
days_since_recorded,
2. feature 3 (0.048662)
population,
3. feature 99 (0.036060)
waterpoint_type_hand pump,
4. feature 87 (0.033346)
quantity_dry,
5. feature 1 (0.030632)
amount_tsh,
6. feature 0 (0.022877)
id,
7. feature 40 (0.022102)
basin_Lake Victoria,
8. feature 61 (0.021326)
construction_year_10s,
9. feature 36 (0.017836)
basin_Internal,
10. feature 56 (0.017836)
scheme_management_wug,
11. feature 59 (0.015316)
permit_Unknown,
12. feature 101 (0.014541)
waterpoint_type_other,
13. feature 71 (0.014153)
management_group_user-group,
14. feature 88 (0.013765)
quantity_enough,
15. feature 97 (0.013571)
waterpoint_type_communal standpipe multiple,
16. feature 76 (0.013571)
payment_pay per bucket,
17. feature 92 (0.012408)
source_class_groundwater,
18. feature 96 (0.012214)
waterpoint_type_communal standpipe,
19. feature 72 (0.012020)
payment_never pay,
20. feature 53 (0.012020)
scheme_management_wtr_auth,
21. feature 78 (0.011826)
payment_unkno

In [26]:
predictions=pd.DataFrame(predictions)
predictions

,0
0,functional
1,functional
2,functional
3,non functional
4,functional
5,functional
6,functional
7,non functional
8,functional
9,functional


In [27]:
#https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/test_set_values.csv
pred_id = pd.read_csv('https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/test_set_values.csv', usecols=['id'])
pred_id

,id
0,50785
1,51630
2,17168
3,45559
4,49871
5,52449
6,24806
7,28965
8,36301
9,54122


In [28]:
final=pd.concat([pred_id,predictions],axis=1)
final

,id,0
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
5,52449,functional
6,24806,functional
7,28965,non functional
8,36301,functional
9,54122,functional


In [29]:
final.columns=['id','status_group']
final

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
5,52449,functional
6,24806,functional
7,28965,non functional
8,36301,functional
9,54122,functional


In [0]:
final.to_csv('predictions_XGB.csv', index=False)
